In [3]:
#!conda install -c menpo dlib -y 
import dlib
import cv2
import numpy as np
from helpers import *
from scipy import ndimage
from imutils.video import VideoStream
from imutils import resize

### Detect Face and Eyes

In [5]:
fce_cascade = cv2.CascadeClassifier('../data/model/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('../data/model/haarcascade_eye.xml')

# This is set to 1 rather than 0 because of switching webcams
vs = VideoStream(1).start()

while True:
    # Read in a single frame
    frame = vs.read()
    # Resize the video window to be smaller
    img = resize(frame, width=600)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = fce_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
        
    cv2.imshow('img',img)
        # If 'Q' is pressed them terminate the video window
    key =  cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        vs.stop()
        cv2.destroyAllWindows()
        break

### Detect all Facial Structure

In [7]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("../data/model/shape_predictor_68_face_landmarks.dat")

# This is set to 1 rather than 0 because of switching webcams
vs = VideoStream(1).start()

while True:
    # Read in a single frame
    frame = vs.read()
    # Resize the video window to be smaller
    frame = resize(frame, width=600)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    clahe_image = clahe.apply(gray)
    detections = detector(clahe_image, 1) #Detect the faces in the image
    for k,d in enumerate(detections): #For each detected face
        shape = predictor(clahe_image, d) #Get coordinates
        #There are 68 landmark points on each face
        for i in range(1,68): 
            #For each point, draw a red circle with thickness2 on the original frame
            cv2.circle(frame, (shape.part(i).x, shape.part(i).y), 1, (0,0,255), thickness=2) 
    cv2.imshow('Finding Facial Features', frame)
    
        # If 'Q' is pressed them terminate the video window
    key =  cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        vs.stop()
        cv2.destroyAllWindows()
        break

### Create Sunglasses Filter

In [6]:
#Combine an image that has a transparency alpha channel
def blend_transparent(face_img, sunglasses_img):

    overlay_img = sunglasses_img[:,:,:3]
    overlay_mask = sunglasses_img[:,:,3:]
    
    background_mask = 255 - overlay_mask

    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

#Find the angle between two points
def angle_between(point_1, point_2):
    angle_1 = np.arctan2(*point_1[::-1])
    angle_2 = np.arctan2(*point_2[::-1])
    return np.rad2deg((angle_1 - angle_2) % (2 * np.pi))


#Start main program
def run(img, glasses, detector, predictor):
    img_copy = img.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    try:
        # detect faces
        dets = detector(gray, 1)

        #find face box bounding points
        for d in dets:

            x = d.left()
            y = d.top()
            w = d.right()
            h = d.bottom()

        dlib_rect = dlib.rectangle(x, y, w, h)

        ##############   Find facial landmarks   ##############
        detected_landmarks = predictor(gray, dlib_rect).parts()

        landmarks = np.matrix([[p.x, p.y] for p in detected_landmarks])

        for idx, point in enumerate(landmarks):
            pos = (point[0, 0], point[0, 1])
            if idx == 0:
                eye_left = pos
            elif idx == 16:
                eye_right = pos

            try:
                #cv2.line(img_copy, eye_left, eye_right, color=(0, 255, 255))
                degree = np.rad2deg(np.arctan2(eye_left[0] - eye_right[0], eye_left[1] - eye_right[1]))

            except:
                pass

        ##############   Resize and rotate glasses   ##############

        #Translate facial object based on input object.

        eye_center = (eye_left[1] + eye_right[1]) / 2

        #Sunglasses translation
        glass_trans = int(.2 * (eye_center - y))

        #Funny tanslation
        #glass_trans = int(-.3 * (eye_center - y ))

        # Mask translation
        #glass_trans = int(-.8 * (eye_center - y))


        # resize glasses to width of face and blend images
        face_width = w - x

        # resize_glasses
        glasses_resize = resize(glasses, face_width)

        # Rotate glasses based on angle between eyes
        yG, xG, cG = glasses_resize.shape
        glasses_resize_rotated = ndimage.rotate(glasses_resize, (degree+90))
        glass_rec_rotated = ndimage.rotate(img[y + glass_trans:y + yG + glass_trans, x:w], (degree+90))


        #blending with rotation
        h5, w5, s5 = glass_rec_rotated.shape
        rec_resize = img_copy[y + glass_trans:y + h5 + glass_trans, x:x + w5]
        blend_glass3 = blend_transparent(rec_resize , glasses_resize_rotated)
        img_copy[y + glass_trans:y + h5 + glass_trans, x:x+w5 ] = blend_glass3
        return img_copy

    except:
        return img_copy

In [7]:
glasses = cv2.imread("../data/img/sunglasses.png", -1)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("../data/model/shape_predictor_68_face_landmarks.dat")

# This is set to 1 rather than 0 because of switching webcams
vs = VideoStream(1).start()

while True:
    # Read in a single frame
    frame = vs.read()
    # Resize the video window to be smaller
    frame = resize(frame, width=600)

    # Finding facial features
    output = run(frame, glasses, detector, predictor)
    cv2.imshow('Glasses Filter', output)
    
    # If 'Q' is pressed them terminate the video window
    key =  cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        vs.stop()
        cv2.destroyAllWindows()
        break